In [2]:
import random
import json

import pymongo
import bson


client = pymongo.MongoClient('localhost', 27017)
db = client.splendor

In [ ]:
# set up new mongodb install
if db.cards.count_documents({}) is 0:
    

In [18]:
# new game
game = db.games.insert_one({"turns_taken": 0})
game_id = game.inserted_id

# get lists from ground truth collections
cards_gt = db.cards.find({"game_id": {"$exists": False}}, {"_id": 0})
tokens_gt = db.tokens.find({"game_id": {"$exists": False}}, {"_id": 0})
tiles_gt = db.tiles.find({"game_id": {"$exists": False}}, {"_id": 0})

# re-insert all the cards with a game ID
game_cards = [dict(i, game_id=game_id) for i in cards_gt]
game_tokens = [dict(i, game_id=game_id) for i in tokens_gt]
all_game_tiles = [dict(i, game_id=game_id) for i in tiles_gt]
game_tiles = random.sample(all_game_tiles, 4)  # only need four for a game

cards = db.cards.insert_many(game_cards)
tiles = db.tiles.insert_many(game_tiles)
tokens = db.tokens.insert_many(game_tokens)

# create players
players = db.players.insert_many([{"game_id": game_id, "order": i} for i in range(3)])

In [19]:
print(db.cards.distinct('game_id'))
print(db.games.distinct('_id'))

[ObjectId('5d8aba050e75bca329caed9e')]
[ObjectId('5d8aba050e75bca329caed9e')]


In [21]:
# set number of players
player_num = 3

# set tokens appropriately
if player_num in [2, 3] and db.tokens.count_documents({'game_id': game_id, 'gem': 'onyx'}) > 5:
    gems = db.tokens.distinct('gem')
    del gems[gems.index('gold')]
    for gem in gems:
        db.tokens.delete_one({'game_id': game_id, 'gem': gem})
        db.tokens.delete_one({'game_id': game_id, 'gem': gem})
        if player_num is 3:
            db.tokens.delete_one({'game_id': game_id, 'gem': gem})

# show four cards
cards = list(db.cards.find({"game_id": game_id}))
show_cards = []
for level in db.cards.distinct('level'):
    l = list(filter(lambda x: x['level'] == level, cards))
    s = random.sample(l, 4)
    show_cards.extend(s)
db.cards.update_many({"_id": {"$in": [n["_id"] for n in show_cards]}}, {"$set": {"in_play": True}})

In [15]:
def drop_all_games(db):
    db.cards.delete_many({"game_id": {"$exists": True}})
    db.tiles.delete_many({"game_id": {"$exists": True}})
    db.tokens.delete_many({"game_id": {"$exists": True}})
    db.players.delete_many({})
    db.games.delete_many({})

drop_all_games(db)